In [1]:
import k3d
import numpy as np

import sys
sys.path.insert(0, "..")

from Geometry.SM import SM

In [5]:
# Choose the super_module from: 'bottom', 'barrel', 'top' by changing the index at the end of this line:
super_module = {0:'bottom', 1:'barrel', 2:'top'}[2]

sm = SM(super_module, kind=super_module)
# to keep mpmt numbering consistent with the full WCTE detector, we need to add an offset
first_mpmt = {'bottom':0, 'barrel':21, 'top':85}[super_module]
first_cam = {'bottom':0, 'barrel':0, 'top':4}[super_module]
first_target = {'bottom':0, 'barrel':0, 'top':16}[super_module]

In [7]:
# Show numbering and orientation of all mPMTs in the WCTE detector super modules

plot = k3d.plot()

# mpmt coordinates
color_z = 0x0000ff
color_x = 0xff0000
color_y = 0x00ff00
origins = []
z_vecs = []
x_vecs = []
y_vecs = []
vec_length = 100. # length of x,z vectors to show for each mpmt

color_mpmt = 0xabb2b9
color_missing_mpmt = 0xa83273
missing_mpmts = [85, 91, 99]
color_camera = 0xed4ef2
color_target = 0xad4113

n_point_mpmt = 8
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])
    
n_fd_point_mpmt = 40
fd_indices_mpmt = []
for i in range(n_fd_point_mpmt):
    fd_indices_mpmt.append([i,(i+1)%n_fd_point_mpmt])

n_point_ft = 20 # for feedthroughs and C holes
indices_ft = []
for i in range(n_point_ft):
    indices_ft.append([i,(i+1)%n_point_ft])

# draw the extent of the mpmt baseplates
for i_mpmt,mpmt in enumerate(sm.mpmts):

    p = mpmt.get_placement('design')
    location, direction_x, direction_z = p['location'], p['direction_x'], p['direction_z']
    direction_y = np.cross(direction_z,direction_x)
    # lists to show mPMT coordinate systems
    z_vec = list(np.array(direction_z)*vec_length)
    x_vec = list(np.array(direction_x)*vec_length)
    y_vec = list(np.array(direction_y)*vec_length)
    origins.append(location)
    if i_mpmt+first_mpmt not in missing_mpmts:
        z_vecs.append(z_vec)
        x_vecs.append(x_vec)
        y_vecs.append(y_vec)

    baseplate_points = np.array(mpmt.get_xy_points('design'),dtype=np.float32)
    color = color_mpmt
    if i_mpmt+first_mpmt in missing_mpmts:
        color = color_missing_mpmt
        
    indices = indices_mpmt
    if mpmt.kind == 'FD':
        indices = fd_indices_mpmt
    plt_baseplate = k3d.lines(baseplate_points, indices, indices_type='segment', color=color)
    plot += plt_baseplate

    if i_mpmt+first_mpmt not in missing_mpmts:
        if mpmt.kind != 'FD':
            feedthrough_points = np.array(mpmt.get_xy_points('design', feature='feedthrough'),dtype=np.float32)
            plt_feedthrough = k3d.lines(feedthrough_points, indices_ft, indices_type='segment', color=color_mpmt)
            plot += plt_feedthrough
        else:
            feedthrough1_points = np.array(mpmt.get_xy_points('design', feature='fd_feedthrough1'),dtype=np.float32)
            plt_feedthrough1 = k3d.lines(feedthrough1_points, indices_ft, indices_type='segment', color=color_mpmt)
            plot += plt_feedthrough1
            feedthrough2_points = np.array(mpmt.get_xy_points('design', feature='fd_feedthrough2'),dtype=np.float32)
            plt_feedthrough2 = k3d.lines(feedthrough2_points, indices_ft, indices_type='segment', color=color_mpmt)
            plot += plt_feedthrough2

    # k3d complains about the following not being float32!
    plt_text = k3d.text(str(i_mpmt+first_mpmt), position=location, reference_point='cc', size=1., label_box=False, color=color)
    plot += plt_text

    # draw the survey C holes
    if 1==1 and i_mpmt+first_mpmt not in missing_mpmts:
        for c in ['c1','c2','c3','c4']:
            survey_points = np.array(mpmt.get_xy_points('design', feature='survey_'+c),dtype=np.float32)
            plt_survey = k3d.lines(survey_points, indices_ft, indices_type='segment', color=color_mpmt)
            plot += plt_survey
            # remove the text for now, it's too cluttered
            #plt_text = k3d.text(c, position=survey_points[0], reference_point='cc', size=1., label_box=False, color=color_mpmt)
            #plot += plt_text

    # draw the fiducial points
    if 1==2 and i_mpmt+first_mpmt not in missing_mpmts:
        fiducial_points = np.array(mpmt.get_fiducials('design'),dtype=np.float32)
        for i_f, fiducial_point in enumerate(fiducial_points):
            text_location = [fiducial_point[j] - 10.*direction_z[j] for j in range(3)]
            plt_text = k3d.text('f'+str(i_f+1), position=text_location, reference_point='cc', size=1., label_box=False, color=color_mpmt)
            plot += plt_text
    
        fiducial_locations = np.array(fiducial_points, dtype=np.float32)
        plt_fps = k3d.points(positions=fiducial_locations,
                            point_size=8.,
                            shader='3d',
                            color=color_mpmt)
        plot += plt_fps

# draw the cameras
if 1==1 and sm.cameras is not None:
    for i_cam,camera in enumerate(sm.cameras):
        dome_points = np.array(camera.get_xy_points('design', feature='dome'),dtype=np.float32)
        plt_dome = k3d.lines(dome_points, indices_ft, indices_type='segment', color=color_camera)
        plot += plt_dome
        
        front_points = np.array(camera.get_xy_points('design', feature='front_housing'),dtype=np.float32)
        plt_front = k3d.lines(front_points, indices_ft, indices_type='segment', color=color_camera)
        plot += plt_front
        
        back_points = np.array(camera.get_xy_points('design', feature='back_housing'),dtype=np.float32)
        plt_back = k3d.lines(back_points, indices_ft, indices_type='segment', color=color_camera)
        plot += plt_back
        
        p = camera.get_placement('design')
        location, direction_x, direction_z = p['location'], p['direction_x'], p['direction_z']
        plt_text = k3d.text(str(i_cam+first_cam), position=location, reference_point='cc', size=1., label_box=False, color=color_camera)
        plot += plt_text
        
        direction_y = np.cross(direction_z,direction_x)
        z_vec = list(np.array(direction_z)*vec_length)
        x_vec = list(np.array(direction_x)*vec_length)
        y_vec = list(np.array(direction_y)*vec_length)
        origins.append(location)
        z_vecs.append(z_vec)
        x_vecs.append(x_vec)
        y_vecs.append(y_vec)
        
        # draw some of the fiducial points
        if 1==1:
            fiducial_points = np.array(camera.get_fiducials('design'),dtype=np.float32)
        
            fiducial_locations = np.array(fiducial_points[0:3], dtype=np.float32)
            plt_fps = k3d.points(positions=fiducial_locations,
                                point_size=8.,
                                shader='3d',
                                color=color_camera)
            plot += plt_fps
        
        
# draw the targets
target_origins = []
if 1==1 and sm.targets is not None:
    for i_target, target in enumerate(sm.targets):
        
        p = target.get_placement('design')
        location, direction_x, direction_z = p['location'], p['direction_x'], p['direction_z']
        target_origins.append(location)
        text_location = [location[j] - 10.*direction_z[j] for j in range(3)]
        plt_text = k3d.text(str(i_target+first_target), position=text_location, reference_point='cc', size=1., label_box=False, color=color_target)
        plot += plt_text

#plot target locations and axis
target_locations = np.array(target_origins, dtype=np.float32)
plt_targets = k3d.points(positions=target_locations,
                        point_size=8.,
                        shader='3d',
                        color=color_target)
plot += plt_targets


plt_z_vecs = k3d.vectors(origins=origins, vectors=z_vecs, color=color_z, head_size=250.)
plot += plt_z_vecs
plt_x_vecs = k3d.vectors(origins=origins, vectors=x_vecs, color=color_x, head_size=250.)
plot += plt_x_vecs
plt_y_vecs = k3d.vectors(origins=origins, vectors=y_vecs, color=color_y, head_size=250.)
plot += plt_y_vecs

plot.display()

Output()

In [ ]:
sm.mpmts[10].get_placement('design')